# graph.packet_registry

> TODO fill in description

In [ ]:
#| default_exp graph.packet_registry

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
from typing import Type, List, Dict, Set
from datetime import datetime, timezone

import fbdev
from fbdev.comp.packet import BasePacket, Packet, PacketUUID
from fbdev.comp.port import PortType,  PortID

In [ ]:
#|export
LocationUUID = str

In [ ]:
#|hide
show_doc(fbdev.graph.packet_registry.TrackedPacket)

---

### TrackedPacket

>      TrackedPacket (packet:BasePacket, location:LocationUUID,
>                     packet_registry:PacketRegistry)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|exporti
class TrackedPacket(BasePacket):
    EXTERNAL_LOCATION = None
    
    def __init__(self, packet:BasePacket, location:LocationUUID, packet_registry:PacketRegistry):
        self._packet = packet
        self._packet_registry = packet_registry
        self._packet_registry._register_packet(self, location)
        
    @property
    def uuid(self) -> PacketUUID: return self._packet.uuid
    @property
    def creation_timestamp(self) -> datetime: return self._packet.creation_timestamp
    @property
    def dtype(self) -> Type: return self._packet.dtype
    @property
    def is_consumed(self) -> bool: return self._packet.is_consumed
    
    async def consume(self):
        data = await self._packet.consume()
        self._packet_registry.register_consumption(self, type(data))
        return data
    
    async def peek(self):
        return await self._packet.peek()
    
    @classmethod
    def get_empty(cls) -> TrackedPacket:
        empty_packet = Packet.get_empty()
        return TrackedPacket(empty_packet)

In [ ]:
#|hide
show_doc(fbdev.graph.packet_registry.PacketActivity)

---

### PacketActivity

>      PacketActivity (packet_uuid:PacketUUID, location:LocationUUID,
>                      timestamp:datetime=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketActivity:
    def __init__(self, packet_uuid: PacketUUID, location: LocationUUID, timestamp: datetime = None):
        self._packet_uuid:PacketUUID = packet_uuid
        if timestamp is None: timestamp = datetime.now(timezone.utc)
        self._timestamp: datetime = timestamp
        self._location: LocationUUID = location
        
    @property
    def packet_uuid(self) -> PacketUUID: return self._packet_uuid
    @property
    def timestamp(self) -> datetime: return self._timestamp
    @property
    def location(self) -> LocationUUID: return self._location
    @property
    def is_location_external(self) -> bool: return self.location == TrackedPacket.EXTERNAL_LOCATION
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(packet_uuid={self.packet_uuid}, location={self.location}, timestamp={self.timestamp})"

#|export
class PacketCreation(PacketActivity):
    def __init__(self, packet_uuid: PacketUUID, location:LocationUUID, packet_dtype: Type, timestamp: datetime = None):
        super().__init__(packet_uuid, location, timestamp=timestamp)
        self._packet_dtype:Type = packet_dtype
    
    @property
    def packet_dtype(self) -> Type: return self._packet_dtype
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(packet_uuid={self.packet_uuid}, location={self.location}, timestamp={self.timestamp})"
    
#|export
class PacketConsumption(PacketActivity):
    def __init__(self, packet_uuid: PacketUUID, location:LocationUUID, payload_dtype: Type, timestamp: datetime = None):
        super().__init__(packet_uuid, location, timestamp=timestamp)
        self._payload_dtype: Type = payload_dtype
    
    @property
    def payload_dtype(self) -> Type: return self._payload_dtype
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(packet_uuid={self.packet_uuid}, location={self.location}, timestamp={self.timestamp})"
        
#|export
class PacketMovement(PacketActivity):
    def __init__(self, packet_uuid: PacketUUID, location: LocationUUID, dest: LocationUUID, via: PortID, timestamp: datetime = None):
        super().__init__(packet_uuid, location, timestamp=timestamp)
        self._dest:LocationUUID = dest
        self._port_id:PortID = via
        
    @property
    def origin(self) -> LocationUUID: return self._location
    @property
    def dest(self) -> LocationUUID: return self._dest
    @property
    def port_id(self) -> PortID: return self._port_id
    @property
    def via(self) -> LocationUUID: return self.port_id
    @property
    def port_type(self) -> PortType: return self._port_id[0]
    @property
    def port_name(self) -> str: return self._port_id[1]
    
    @property
    def is_origin_external(self) -> bool: return self.origin == TrackedPacket.EXTERNAL_LOCATION
    @property
    def is_dest_external(self) -> bool: return self.dest == TrackedPacket.EXTERNAL_LOCATION
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(packet_uuid={self.packet_uuid}, origin={self.origin}, dest={self.dest}, via={self.via}, timestamp={self.timestamp})"


In [ ]:
#|hide
show_doc(fbdev.graph.packet_registry.PacketRegistry)

---

### PacketRegistry

>      PacketRegistry ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketRegistry:
    def __init__(self):
        self._packets:Dict[PacketUUID, TrackedPacket] = {}
        self._consumed_packets:Set[PacketUUID] = set()
        self._packet_locations:Dict[PacketUUID, LocationUUID] = {}
        self._history: List[PacketActivity] = []
    
    def is_registered(self, packet:TrackedPacket) -> bool:
        return packet.uuid in self._packets
    
    def _register_packet(self, packet:TrackedPacket, location:LocationUUID|TrackedPacket.EXTERNAL_LOCATION):
        """This method should only be called within the constructor of TrackedPacket."""
        if type(packet) != TrackedPacket:
            raise TypeError(f"Argument `packet` must be of type `TrackedPacket`, got '{type(packet)}'.")
        if packet.uuid in self._packets:
            loc_address = fbdev.graph.net.lookup_location_uuid(self._packet_locations[packet.uuid]).address
            raise RuntimeError(f"Tried registering packet '{packet.uuid}' at {loc_address} already exists in registry.")
        self._packets[packet.uuid] = packet
        self._packet_locations[packet.uuid] = location
        self._history.append(
            PacketCreation(packet.uuid, location, packet.dtype, timestamp=packet.creation_timestamp)
        )
    
    def register_move(self, packet:TrackedPacket, origin:LocationUUID, dest:LocationUUID, via:PortID):
        if packet.uuid not in self._packets:
            raise RuntimeError(f"Packet '{packet.uuid}' is not registered.")
        if origin != self._packet_locations[packet.uuid]:
            origin_address = fbdev.graph.net.lookup_location_uuid(origin).address
            dest_address = fbdev.graph.net.lookup_location_uuid(dest).address
            raise RuntimeError(f"Tried to move packet '{packet.uuid}' from '{origin_address}' to '{dest_address}', but it is not at '{origin_address}'.")
        self._history.append(
            PacketMovement(packet.uuid, origin, dest, via)
        )
        
    def register_consumption(self, packet:TrackedPacket, payload_dtype:Type):
        if packet.uuid not in self._packets:
            raise RuntimeError(f"Packet '{packet.uuid}' is not registered.")
        if not packet.is_consumed:
            loc_address = fbdev.graph.net.lookup_location_uuid(self._packet_locations[packet.uuid]).address
            raise RuntimeError(f"Tried to register packet '{packet.uuid}' as consumed at '{loc_address}', but it is already consumed.")
        self._consumed_packets.add(packet.uuid)
        self._history.append(
            PacketConsumption(packet.uuid, self._packet_locations[packet.uuid], payload_dtype)
        )